In [ ]:
# Install fastai
!pip install fastai -q
!pip install fastai --upgrade

In [2]:
from fastai.tabular.all import *
import pandas as pd
import csv

In [13]:
# Load the csv as a dataframe
path = Path(".")
df1 = pd.read_csv(path/'merged_df1.csv')
# Remove the id column from R's dataframe
del df1['Unnamed: 0']

In [14]:
df1.head

<bound method NDFrame.head of     age    sex   city  ... chest pain diarrhoea physical discomfort
0     1  6a50b  7a6d2  ...      False     False               False
1    40  6a50b  593f0  ...      False     False               False
2    56  8ef1a  17787  ...      False     False               False
3    44  6a50b  a3f17  ...      False     False               False
4    31  8ef1a  8538a  ...      False     False               False
..   ..    ...    ...  ...        ...       ...                 ...
633  75  8a467  d7cac  ...      False     False               False
634  80  1e6ee  c4d2f  ...      False     False               False
635  55  8a467  bc45c  ...      False     False               False
636  57  d516d  73c3e  ...      False     False               False
637  51  8a467  56626  ...      False     False               False

[638 rows x 86 columns]>

In [15]:
print(df1)

    age    sex   city  ... chest pain diarrhoea physical discomfort
0     1  6a50b  7a6d2  ...      False     False               False
1    40  6a50b  593f0  ...      False     False               False
2    56  8ef1a  17787  ...      False     False               False
3    44  6a50b  a3f17  ...      False     False               False
4    31  8ef1a  8538a  ...      False     False               False
..   ..    ...    ...  ...        ...       ...                 ...
633  75  8a467  d7cac  ...      False     False               False
634  80  1e6ee  c4d2f  ...      False     False               False
635  55  8a467  bc45c  ...      False     False               False
636  57  d516d  73c3e  ...      False     False               False
637  51  8a467  56626  ...      False     False               False

[638 rows x 86 columns]


In [18]:
list(df1.columns)[6]

'duration'

In [35]:
# Get a list of categorical columns
df1_cat = list(df1.columns)

df1_cat = df1_cat[0:6] + df1_cat[8: ]
# Only age and duration are numerical, the others are categorical

'physical discomfort'

In [32]:
# Get a list of numerical columns, not counting the thing we're predicting
df1_num = list(df1.columns)[7:8]
# The only numerical columns are the age and duration
# We don't include duration because we are predicting it
list(df1.columns)[7]
# This column should be about age

'clean_age'

In [36]:
# Use the fast.ai tabular dataloader

dls = TabularDataLoaders.from_csv(
    path/'merged_df1.csv', y_names = "duration",
    cat_names = df1_cat, cont_names = df1_num,
    procs = [Categorify, FillMissing, Normalize]
    )

learn = tabular_learner(dls, y_range = (0, 33),  metrics = accuracy)

In [ ]:
# I determined the y_range by using
# ```max(merged_df1$duration)```
# in R and adding one to the maximum. I remember it being recommended
# by the fast.ai prof, I think because the neuron is asymptotic to
# the maximum value, I'm not entirely sure.

In [38]:
# Training and validation splits
splits = RandomSplitter(valid_pct=0.2)(range_of(df1))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [26]:
to = TabularPandas(df1, procs = [Categorify, FillMissing, Normalize],
                   cat_names = df1_cat, cont_names = df1_num,
                   y_names = 'duration',
                   splits = splits)

dls = to.dataloaders(bs = 16)

In [64]:
learn = tabular_learner(dls, y_range = (0, 32), metrics = rmse)

In [68]:
learn.fit_one_cycle(20, lr_max = 0.005)

epoch,train_loss,valid_loss,_rmse,time
0,0.763763,18.027575,4.245889,00:00
1,0.761995,20.238358,4.498707,00:00
2,0.969725,17.785206,4.217251,00:00
3,1.587254,27.656521,5.258947,00:00
4,2.705521,17.337973,4.163889,00:00
5,3.757890,68.032791,8.248199,00:00
6,4.357812,25.065008,5.006497,00:00
7,4.434713,19.815388,4.451448,00:00
8,4.304705,81.181793,9.010093,00:00
9,4.123631,33.105320,5.753723,00:00


In [69]:
learn.predict(learn.dls.train.all_cols.iloc[0])

(   age  sex  city  ...  clean_age_na  clean_age  duration
 0  0.0  0.0   0.0  ...           1.0  -2.679651  2.259895
 
 [1 rows x 87 columns], tensor([2.2599]), tensor([2.2599]))

In [70]:
learn.export()

In [72]:
df2 = pd.read_csv(path/'test2_cleaned.csv')

In [74]:
splits2 = RandomSplitter(valid_pct=0.2)(range_of(df2))

In [75]:
to2 = TabularPandas(df2, procs = [Categorify, FillMissing, Normalize],
                    cat_names = df1_cat, cont_names = df1_num,
                    y_names = 'duration',
                    splits = splits2)

dls2 = to2.dataloaders(bs = 16)

KeyError: ignored

In [ ]:
learn = tabular_learner(dls, y_range = (0, 33), metrics = rmse)

In [ ]:
test_df = df2
test_df.drop(['duration'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

In [ ]:
p1 = learn.get_preds(dl = dl)

In [ ]:
p2 = p1[0].tolist()

In [ ]:
 with open('testing.csv', mode='w') as employee_file:
   employee_writer = csv.writer(employee_file, delimiter = ',',
                                quotechar = '"', quoting = csv.QUOTE_MINIMAL)
   employee_writer.writerow(p2)
print("Done")

Done
